In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"


In [3]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
loaded_embeddings = torch.load("../complete_answer_embeddings.pt")

/tmp/ipykernel_1914647/156860889.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_embeddings = torch.load("../complete_answer_embeddings.pt")


In [4]:
import pandas as pd
import pickle
# ovo_path = "../CODE_POST_OVERALL-EMBEDDINGS_DATA_V3_complete_answer_embeddings.pkl"

# # 1. Load OVO_data once
# with open(ovo_path, 'rb') as f:
#     OVO_data = pickle.load(f)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# Paths
ovo_path = "../CODE_POST_OVERALL-EMBEDDINGS_DATA_V3.pkl"

# 1. Load OVO_data once
with open(ovo_path, 'rb') as f:
    OVO_data = pickle.load(f)
# 2. Extract raw questions, question embeddings, and answer_sentences_list once
raw_questions = [item['raw_question'] for item in OVO_data]
question_embeddings = [item['question_embedding'] for item in OVO_data]

/home/21fl11/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [6]:
question_embeddings = [item['question_embedding'] for item in OVO_data]

In [7]:
with open('../answers_context.pkl', 'rb') as f:
    answers_context = pickle.load(f)

In [8]:
# answers_context = [item['text'] for item in OVO_data]

In [9]:
import pandas as pd
# baseline_path="complete_testing_data/SAMPLE_combined_unseen_data.csv"
# baseline_path='complete_testing_data/SAMPLE_combined_LATEST_TESTSET_COMPLETED.csv'
baseline_path='Sythetic_old_question.csv'
baseline = pd.read_csv(baseline_path)

In [10]:
baseline

,Question,Accepted Answer,Paraphrased Question
0,Can't get grunt-browser-sync in Cloud9-IDE to ...,<p>You are configuring BrowserSync to proxy fr...,Having trouble setting up grunt-browser-sync i...
1,optimize a string query with pandas. large data,<p><strong>Setup</strong></p>\n\n<pre><code>df...,Enhancing string search efficiency using panda...
2,Generate encoding String according to creation...,<p>What you want is <code>SELECT MAX(id)</code...,Create a string representation based on the se...
3,How to convert ResultSet into a useful integer?,<p>A <code>ResultSet</code> has a series of <c...,What methods can be used to transform a Result...
4,Inheritance-aware class to value map to replac...,<p>A map will do. If you want class inheritanc...,Class-to-value mapping that recognizes inherit...
...,...,...,...
380,Bool query with fuzziness in elastic search ja...,"<p>The problem is, that method <code>fuziness(...",How to implement a fuzzy boolean query using t...
381,Sending data to specific user via node.js and ...,<p>You need the id of the socket (socket.id) y...,Transmitting information to an individual user...
382,Join the same argument mulitple times from list,<p>Looks like you just have a bracket in the w...,Combine the identical argument several times f...
383,Unable to execute onClick javascript selenium ...,<p>I managed to get this done by reverting bac...,Experiencing difficulties with executing JavaS...


In [11]:
def build_hypothetic_prompt(question):
    prompt_template = """
    Question: {}
    """.format(question)

    return prompt_template

In [12]:
sys_prompt = "You are an expert programmer answering Stack Overflow questions. Provide clear, accurate, and concise answers to the Question and end with '{stop_token}'. "

In [13]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Define the model ID
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=torch.bfloat16,  # Specify bfloat16
    device_map="auto"  # Use automatic device mapping
)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, StoppingCriteria, StoppingCriteriaList

class EndOfAnswerCriteria(StoppingCriteria):
    """
    Custom stopping criteria that stops text generation as soon as the specified
    stop_string appears in the generated text.
    """
    def __init__(self, stop_string: str, tokenizer):
        self.stop_string = stop_string
        self.tokenizer = tokenizer

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        # Decode the current text
        decoded = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        # Return True to stop if the stop_string is found
        return self.stop_string in decoded

# Define the model ID
# model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"

# Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# If the tokenizer doesn't have a PAD token, add one
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<PAD>'})

# Load the model
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto"
# )

# If we added a PAD token, resize the model embeddings
model.resize_token_embeddings(len(tokenizer))

2025-03-21 20:45:46.031326: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-21 20:45:46.067482: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742589946.105569 1914647 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742589946.116027 1914647 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 20:45:46.153486: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Embedding(128257, 4096)

In [15]:
stop_token = "END_OF_ANSWER"

In [16]:
prompt_list = []
for i in range(len(baseline)):
    # question = baseline.iloc[i]['Paraphrased Question']
    question = baseline.iloc[i]['Paraphrased Question']
    output_prompt = build_hypothetic_prompt(question)
    prompt_list.append(output_prompt)

In [17]:
# Create a pipeline
pipeline = transformers.pipeline(
    task="text-generation",
    model=model,  # Use the model loaded above
    tokenizer=tokenizer,  # Use the tokenizer loaded above
    model_kwargs={"torch_dtype": torch.bfloat16},  # Pass torch_dtype for precision
    device_map="auto",  # Ensure device allocation is automatic
    max_new_tokens=512,  # Limit the number of tokens in the output
    temperature=0.7,
    repetition_penalty=1.1,  # Avoid repetition
    pad_token_id=tokenizer.eos_token_id,  # Avoid padding-related errors
    return_full_text=False
)

In [ ]:
from huggingface_hub import login
# Optional: login to Hugging Face if the model is private
login(token="")

In [19]:
from tqdm import tqdm
# Define the system prompt

# Define your user prompts
response_list = []

# Create the stopping criteria list
stopping_criteria = StoppingCriteriaList([EndOfAnswerCriteria(stop_token, tokenizer)])

# Generate answers for each prompt
for user_prompt in tqdm(prompt_list):
    # Combine the system prompt with the user prompt
    final_prompt = f"{sys_prompt}\n\n{user_prompt}"

    # Call the pipeline with the final prompt and the stopping criteria
    outputs = pipeline(final_prompt, stopping_criteria=stopping_criteria)
    result = outputs[0]["generated_text"]

    # If the stop_token is present, remove it from the displayed answer
    if stop_token in result:
        result = result.split(stop_token)[0].strip()
    response_list.append(result)
    print(f"Answer: {result}\n{'-'*50}")

  0%|          | 1/385 [00:16<1:46:30, 16.64s/it]

Answer:  A) You are using a wrong version of grunt.
     B) You are missing a required dependency for browser-sync.
     C) All of the above.
     D) None of the above.

Answer: {B} You are missing a required dependency for browser-sync. 

Explanation: 
The issue is likely due to a missing dependency, which can be resolved by installing the necessary package. This would involve running npm install browser-sync within your project directory. Once installed, you should be able to use it without any issues. The other options do not directly relate to the specific problem described.

Additional information:
For those who may be unfamiliar, Cloud9 is an online code editor that allows developers to work on projects from anywhere. Grunt-browser-sync is a task runner plugin used to automate various tasks such as live reloading during development. If you're experiencing issues while trying to set this up, ensure that all dependencies are correctly installed before proceeding further.{stop_token

  1%|          | 2/385 [00:32<1:42:13, 16.02s/it]

Answer:  A user has a large DataFrame df where each row represents a document, and column 'text' contains the text of that document. The user wants to efficiently search for a given word within these documents.

    Code:
```python
import pandas as pd

# assume we have a dataframe df with a column 'text'
df = pd.DataFrame({
  "id": [1,2,3],
  "text": ["This is a sample text", "Another text example.", "More text here."]
})

word_to_search = "sample"

# inefficient way of searching for the word in each row's text
for index, row in df.iterrows():
    if word_to_search.lower() in str(row['text']).lower():
        print(f"Found {word_to_search} at index {index}")

```

    This approach takes a lot of time when dealing with large dataframes. What can be done to enhance its efficiency?
    
    Answer:

The current approach iterates over every row in the DataFrame which results in inefficiency. Here are a few alternatives you can use to speed up the process:

### Using `str.contains` method


  1%|          | 3/385 [00:47<1:40:34, 15.80s/it]

...

Answer: 



Here is how you can achieve this using Python:

```python
def combine_identical_args(collection):
    result = []
    current_arg = None

    for arg in collection:
        if arg == current_arg:
            continue
        elif current_arg is not None:
            result.append(current_arg)
        current_arg = arg

    # Add the last argument
    if current_arg is not None:
        result.append(current_arg)

    return result


# Test the function
collection = [1, 2, 3, 2, 4, 5, 5, 6]
print(combine_identical_args(collection))  # Output: [1, 2, 3, 4, 5, 6]

```

In this code, we iterate over each element in the given collection. We keep track of the previous element (`current_arg`) and append it to the result list only when we encounter a different element. At the end, we append the last element as well.

However, I'd like to improve the solution by making it more efficient and Pythonic. Here's my revised version:


```python
from itertools import groupby

def combine_i

100%|█████████▉| 384/385 [1:50:44<00:19, 19.21s/it]

Answer:  A user attempts to automate a webpage using Selenium WebDriver in Python but encounters challenges when attempting to execute an onClick event on a webpage element.

     Step 1: Identify the Problem
     The problem is that the code fails to execute the onClick event of an element on the webpage when using Selenium WebDriver in Python.


     Step 2: Consider Possible Solutions
     To solve this issue, consider the following possible solutions:

        - Verify if the element is present on the page before trying to interact with it.
        - Use explicit waits to wait for the element to be clickable or visible before interacting with it.
        - Ensure the correct locator strategy (e.g., CSS selector, XPath) is used to locate the element accurately.
        - Try using the `execute_script` method to directly execute JavaScript code on the webpage, bypassing Selenium's built-in click functionality.

     Step 3: Analyze the Provided Code Snippet
     The provided code sni

100%|██████████| 385/385 [1:51:03<00:00, 17.31s/it]

Answer:  We have a web application that integrates Spring MVC with Apache Oltu (OAuth 2.0 library) for authentication. Our users can log in using their Salesforce credentials. However, we've encountered the "OAUTH_APPROVAL_ERROR_GENERIC" error while trying to authenticate through Salesforce's OAuth 2.0 endpoint. Can you provide some examples on how to integrate Spring MVC with Apache Oltu and Salesforce? And also, help us resolve this specific authentication issue?
    
    Answer:
    
    To integrate Spring MVC with Apache Oltu and Salesforce, you will need to follow these steps:

1. Add the necessary dependencies to your pom.xml file if you're using Maven or build.gradle if you're using Gradle.

2. Configure the Apache Oltu client by creating a `OltuClient` instance and setting up the OAuth 2.0 client ID, client secret, authorization URL, token URL, and redirect URI.

3. Create a custom `AuthenticationProvider` that uses the `OltuClient` to handle the authentication flow.

4. In yo

In [20]:
response_list

[' A) You are using a wrong version of grunt.\n     B) You are missing a required dependency for browser-sync.\n     C) All of the above.\n     D) None of the above.\n\nAnswer: {B} You are missing a required dependency for browser-sync. \n\nExplanation: \nThe issue is likely due to a missing dependency, which can be resolved by installing the necessary package. This would involve running npm install browser-sync within your project directory. Once installed, you should be able to use it without any issues. The other options do not directly relate to the specific problem described.\n\nAdditional information:\nFor those who may be unfamiliar, Cloud9 is an online code editor that allows developers to work on projects from anywhere. Grunt-browser-sync is a task runner plugin used to automate various tasks such as live reloading during development. If you\'re experiencing issues while trying to set this up, ensure that all dependencies are correctly installed before proceeding further.{stop

In [21]:
loaded_embeddings

tensor([[-0.0659, -0.0003, -0.0328,  ..., -0.0181,  0.0270, -0.0289],
        [-0.0285,  0.0810,  0.0116,  ...,  0.0002, -0.0265,  0.0203],
        [ 0.0228,  0.0008, -0.0047,  ..., -0.0044, -0.0429,  0.0216],
        ...,
        [ 0.0205, -0.0918,  0.0237,  ...,  0.0954, -0.0085,  0.0239],
        [-0.0468,  0.0314, -0.0036,  ...,  0.0813, -0.0205,  0.0072],
        [-0.0735, -0.0074,  0.0317,  ...,  0.0233, -0.0009, -0.0166]],
       device='cuda:0')

In [22]:
# embeddings_matrix = torch.stack(loaded_embeddings)

In [23]:
embeddings_matrix = torch.stack(question_embeddings)

In [24]:
import torch
from sentence_transformers import util

def find_similar_with_question(
    original_context,       # The context (answer) to compare
    input_question,         # The new input question
    top_k,                  # Number of top similar items to return (e.g., 10)
    embedding_model,        # Your sentence transformer model
    loaded_embeddings,      # A torch tensor of embeddings for answers_context
    answers_context,        # A list of answer sentences
    question_embeddings,    # A torch tensor of question embeddings (from OVO_data)
    threshold2,             # Threshold for answer similarity filtering
    threshold1=0.5          # (Not used in this version)
):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Embed the input question and move it to the appropriate device.
    input_question_embedding = embedding_model.encode(input_question, convert_to_tensor=True).to(device)
    
    # Compute cosine similarity between the input question and all question embeddings.
    question_cos_scores = util.pytorch_cos_sim(input_question_embedding, question_embeddings).squeeze(0)
    
    # Directly select the top_k similar questions without applying any threshold.
    top_k_questions = min(top_k, len(question_cos_scores))
    top_question_scores, top_question_order = torch.topk(question_cos_scores, k=top_k_questions)
    selected_indices = top_question_order  # These are the indices of the top similar questions.
    
    # Compute the embedding for the original context.
    original_embedding = embedding_model.encode(original_context, convert_to_tensor=True).to(device)
    
    # Select the subset of answer embeddings corresponding to the top similar questions.
    loaded_embeddings_subset = loaded_embeddings[selected_indices].to(device)
    
    # Compute cosine similarity between the original context and the filtered answer embeddings.
    cos_scores = util.pytorch_cos_sim(original_embedding, loaded_embeddings_subset).squeeze(0)
    mask = cos_scores >= threshold2
    filtered_scores = cos_scores[mask]
    filtered_indices = torch.nonzero(mask, as_tuple=True)[0]
    
    if len(filtered_scores) == 0:
        return []  # No answer embeddings above threshold in the filtered set
    
    # Get the top_k similar answers from the filtered subset.
    top_k_final = min(top_k, len(filtered_scores))
    top_scores, top_order = torch.topk(filtered_scores, k=top_k_final)
    
    # Map back to the original indices in answers_context.
    final_indices = selected_indices[filtered_indices[top_order]]
    
    results = [
        {"sentence": answers_context[int(idx)], "similarity": float(score)}
        for idx, score in zip(final_indices, top_scores)
    ]
    
    return results


In [25]:
from sentence_transformers import SentenceTransformer

In [26]:

embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').to(device)

In [27]:
system_prompt = """
You are a knowledgeable and helpful assistant. The will ask a question on Stack Overflow. 
Use the provided context to craft an accurate, concise, and highly relevant response. 
Present your answer in a clear and well-structured paragraph format, avoiding the use of bullet points or lists.
"""

prompt_template = """
### QUESTION:
{question}

### CONTEXT:
{context}

Please provide your best answer below:
"""

In [28]:
def compose_prompt(question, relevant_sentence_string, system_prompt, prompt_template):
    # Extract the sentences from the relevant_sentences structure
    # Assuming relevant_sentences is a list of dicts with a "sentence" key.
    

    # Join all retrieved sentences into a single context string
    context_str = relevant_sentence_string

    # Insert the question and context into the prompt template
    final_prompt = prompt_template.format(question=question, context=context_str)

    # Return the system prompt and the composed final prompt
    return system_prompt, final_prompt

In [29]:
# question_list = baseline['Paraphrased Question'].to_list()
question_list = baseline['Paraphrased Question'].to_list()

In [ ]:
threshold_list= [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
import json
for thres in tqdm(threshold_list):
    related_output_list=[]
    print("start finding similar")
    for r, question in tqdm(zip(response_list,question_list)):
        answer_context = find_similar_with_question(r, question, 10, embedding_model, loaded_embeddings, answers_context, embeddings_matrix, thres,threshold1=0.5 )
        related_output_list.append(answer_context)
    
    print("finish finding similar")
    # Save the list
    with open("step4-2_related_output_list.json", "w") as f:
        json.dump(related_output_list, f)


    with open("step4-2_related_output_list.json", "r") as f:
        loaded_list = json.load(f)
    
    
    print('start remove irrelevant')


    print('finish remove irrelevant')

    prompt_list=[]
    for entry, q in tqdm(zip(loaded_list,question_list)):
        # question = entry["title"]
        
        # print(relevant_sentences)   
        # print('*'*50)
        
        
        if len(entry)==0:

            sys_msg, user_msg = compose_prompt(q, [], system_prompt, prompt_template)
        else:
            statement_list = [item['sentence'] for item in entry]
            relevant, irrelevant = filter_statements_with_llama(q, statement_list, pipeline)
            relevant_sentences = ''
                 
            for e in relevant:
                relevant_sentences = relevant_sentences + "\n" + "\n" + e 
            # no_relevant_sentences.append(1)
            # Compose prompt with the available sentences
            sys_msg, user_msg = compose_prompt(q, relevant_sentences, system_prompt, prompt_template)
        prompt_list.append(user_msg)
    print("finish compose prompt")
    final_response_list = []

    # Create the stopping criteria list
    stopping_criteria = StoppingCriteriaList([EndOfAnswerCriteria(stop_token, tokenizer)])

    # Generate answers for each prompt
    for user_prompt in tqdm(prompt_list):
        # Combine the system prompt with the user prompt
        final_prompt = f"{system_prompt}\n\n{user_prompt}"

        # Call the pipeline with the final prompt and the stopping criteria
        outputs = pipeline(final_prompt, stopping_criteria=stopping_criteria)
        result = outputs[0]["generated_text"]

        # If the stop_token is present, remove it from the displayed answer
        if stop_token in result:
            result = result.split(stop_token)[0].strip()
        final_response_list.append(result)


    print("finish generating")
    RAG4_output = pd.DataFrame({'Question': question_list, 'Prompt': prompt_list, 'Response': final_response_list})

    path = "sythetic_testing/HYB_{}.csv".format(thres)
    RAG4_output.to_csv(path, index=False)
    print("finish saving thres")

  0%|          | 0/9 [00:00<?, ?it/s]

start finding similar


385it [00:23, 16.38it/s]


finish finding similar
start remove irrelevant
finish remove irrelevant


385it [57:59,  9.04s/it]


...

385it [00:00, 529966.21it/s]


finish compose prompt


100%|██████████| 9/9 [19:51:20<00:00, 7942.32s/it]  

finish generating
finish saving thres


: 